In [21]:
import pandas
from pandas import DataFrame
import re

# Load CSV of Poker Now events
poker_log: DataFrame = pandas.read_csv('../data/poker_now_log_Gnt9_F6MRcTknj6YjZfTNtdOS.csv')
poker_log.drop(columns='order', inplace=True)
# Reverse events so timestamps are in ascending order
poker_log = poker_log[::-1].reset_index(drop=True)

poker_log = poker_log.assign(hand_number=0)
poker_log = poker_log.assign(street='')

starting_hand_rxp = re.compile("-- starting hand #(.*)  \(No Limit Texas Hold'em\) \((.*)\) --")

ending_hand_rxp = re.compile("-- ending hand #(\d) --")

# Hand Numbers
curr_idx = 0
while curr_idx < len(poker_log):
    sliced = poker_log.copy()[curr_idx:]

    # Find start index and end index of starting/ending hands
    # Since the PokerNow Event Log has showdowns between the end of one hand and the start of the next, we must track
    # both ending hand events and all events before the start of the next hand
    start_and_end = sliced.entry.str.startswith('-- starting hand').nlargest(2)
    # If there is no following

    # Starting Hand
    start_hand_idx = start_and_end.index.values[0]

    # Ending Hand Index
    end_hand_idx = sliced.entry.str.startswith('-- ending hand').idxmax()

    # Next Starting Hand
    if start_and_end.iloc[1]:
        next_start_idx = start_and_end.index.values[1]
    else:
        next_start_idx = len(poker_log)
    # Just before next starting hand
    end_idx = next_start_idx - 1

    street_slice = poker_log.copy()[start_hand_idx:end_idx+1]

    if street_slice.entry.str.startswith('Flop:').any():
        flop_idx = street_slice.entry.str.startswith('Flop:').idxmax()
        poker_log.street[start_hand_idx:flop_idx] = 'preflop'
        if street_slice.entry.str.startswith('Turn:').any():
            turn_idx = street_slice.entry.str.startswith('Turn:').idxmax()
            poker_log.street[flop_idx:turn_idx] = 'flop'
            if street_slice.entry.str.startswith('River:').any():
                river_idx = street_slice.entry.str.startswith('River:').idxmax()
                poker_log.street[turn_idx:river_idx] = 'turn'
                poker_log.street[river_idx:end_hand_idx+1] = 'river'
                if end_idx != end_hand_idx:
                    poker_log.street[end_hand_idx+1: end_idx+1] = 'showdown'
            else:
                poker_log.street[turn_idx:end_idx+1] = 'turn'
        else:
            poker_log.street[flop_idx:end_idx+1] = 'flop'
    else:
        # No flop, skip others
        poker_log.street[start_hand_idx:end_idx+1] = 'preflop'

    # Parse hand number using regular expression
    start_match = re.match(starting_hand_rxp, sliced.entry[start_hand_idx])
    # Hand number
    hand_number = int(start_match.group(1))
    # Set hand numbers
    # If start of game or showdown occurred last hand, mark hand number appropriately
    # Else mark hand number as it was parsed from the regexp
    poker_log.hand_number[start_hand_idx:end_idx+1] = hand_number
    curr_idx = next_start_idx

<ipython-input-21-f7aa334441b0>:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-f7aa334441b0>:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-f7aa334441b0>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-21-f7aa334441b0>:54: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [22]:
import plotly.express as px

sot = poker_log[poker_log.entry.str.startswith('Player stacks:')].copy(deep=True)
sot.drop(columns=['street'], inplace=True)

sot.entry = sot.entry.str.lstrip('Player stacks: ')
sot = sot.assign(player_number=0)
sot = sot.assign(player='')
sot = sot.assign(stack_size=0)
sot = sot.assign(split=sot.entry.str.split(' \| ')).explode('split')
STACK_MATCHER = re.compile('#(\d+) \"(.*?)\" \((\d+)\)$')

def parse_stack_size(entry):
    match = STACK_MATCHER.match(entry)
    return match.group(1), match.group(2), match.group(3)

sot['player_number'],  sot['player'], sot['stack_size'] = zip(*sot['split'].apply(parse_stack_size))

sot.drop(columns=['entry', 'split'], inplace=True)
sot = sot.reset_index(drop=True)

fig = px.line(sot, x="at", y="stack_size", color ="player", title ='''Stack Size by Hand''',
        labels={'At':'Time','stack_size':'Chip Amount'}, hover_name='stack_size', hover_data=['stack_size'], width=1000, height=600)
fig.update_layout(
    title={
        'y':0.9,
        'x':0.5,
         'xanchor': 'center',
        })
fig.update_traces(mode='markers+lines')
fig.show()

In [23]:
poker_log = poker_log[~poker_log['entry'].astype(str).str.startswith('Player stacks: ')]

poker_log = poker_log.assign(player='', action='', value='')

YOUR_HAND_MATCHER = re.compile(r'(?<=Your hand is ).*')
SHOWS_HAND_MATCHER = re.compile(r'(?<=shows a ).*')

SMALL_BLIND_MATCHER = re.compile('\"(.*)\" posts a small blind of (\d+)(?=\s|$)')
BIG_BLIND_MATCHER = re.compile('\"(.*)\" posts a big blind of (\d+)(?=\s|$)')

BETS_MATCHER = re.compile('\"(.*)\" bets (\d+)(?=\s|$)')
RAISES_MATCHER = re.compile('\"(.*)\" raises to (\d+)(?=\s|$)')
CALLS_MATCHER = re.compile('\"(.*)\" calls (\d+)(?=\s|$)')
FOLDS_MATCHER = re.compile('\"(.*)\" folds$')
CHECKS_MATCHER = re.compile('\"(.*)\" checks$')
COLLECTED_MATCHER = re.compile('\"(.*)\" collected (\d+) from pot')

JOINED_MATCHER = re.compile('The player \"(.*)\" joined the game with a stack of (\d+)\.')
QUIT_MATCHER = re.compile('The player \"(.*)\" quits the game with a stack of (\d+)\.')

poker_log.player = poker_log.entry.str.extract("\"(.*)\".*")
poker_log.player.fillna('')
def parse_action_value(entry):
    if match := SMALL_BLIND_MATCHER.match(entry):
        return 'small_blind', int(match.group(2))
    elif match := BIG_BLIND_MATCHER.match(entry):
        return 'big_blind', int(match.group(2))
    elif match := BETS_MATCHER.match(entry):
        return 'bets', int(match.group(2))
    elif match := RAISES_MATCHER.match(entry):
        return 'raises', int(match.group(2))
    elif match := CALLS_MATCHER.match(entry):
        return 'calls', int(match.group(2))
    elif CHECKS_MATCHER.match(entry):
        return 'checks', ''
    elif FOLDS_MATCHER.match(entry):
        return 'folds', ''
    elif match := COLLECTED_MATCHER.match(entry):
        return 'collected', int(match.group(2))
    elif match := JOINED_MATCHER.match(entry):
        return 'joined_with_stack', int(match.group(2))
    elif QUIT_MATCHER.match(entry):
        return 'quit', ''
    else:
        return '', ''
poker_log['action'],  poker_log['value'] = zip(*poker_log['entry'].apply(parse_action_value))

In [24]:
from numpy import nan
from numpy import isnan

df = poker_log.copy(deep=True)
df.replace('', nan, inplace=True)
df.action.replace('joined_with_stack', nan, inplace=True)
df.dropna(subset=['action', 'value'], inplace=True)
df.drop(columns='entry', inplace=True)

"""
First find the winner of the hand (whoever collected)
Then create a dict with the names of the participants per street -> last wager in a street
Merge each dict with the wagers on name
Remove winning name from dictionary.
For each hand, we'll have: a dictionary with everyone who lost money from a person
cody: {antoine: 10, gautam: 10}
row = cody, col = antoine, value

create giant df
for each hand_number
    create dict
    for each street:
        reverse
        for each row:
            add player name to dict with value (max(timestamp)
    add dict to giant df
"""

unique_players = poker_log.player.dropna().unique()
total_money_won = DataFrame(columns=unique_players, index=unique_players)
total_money_won.fillna(0)
hand_groups = df.groupby('hand_number', sort=False)
for hand_number, hand_group in hand_groups:
    # Multiple winners if split pot
    winners = set(hand_group[hand_group['action'] == 'collected'].player.values)
    # print(f'Hand Number: {hand_number}, Winner: {winner_name}')
    loser_dict: dict = {}
    street_groups = hand_group.groupby('street')
    for street, street_group in street_groups:
        player_groups = street_group.groupby('player')
        for player, player_group in player_groups:
            if player not in winners:
                # print(f'Player: {player}, Last Play: {player_group.tail(1).value.values[0]} in street: {street}')
                last_play_value = player_group.tail(1).value.values[0]
                if player in loser_dict:
                    loser_dict[player] += last_play_value
                else:
                    loser_dict[player] = last_play_value
    # print(f'Loser Dict: {loser_dict}')
    for loser_name, value in loser_dict.items():
        # print(f'Loser: {loser_name}, Total Lost: {value} in hand: {hand_number}')
        # If split pot, divide by the number of winners
        split_value = value / len(winners)
        for winner_name in winners:
            if not isnan(total_money_won.loc[winner_name, loser_name]):
                total_money_won.loc[winner_name, loser_name] = total_money_won.loc[winner_name, loser_name] + split_value
            else:
                total_money_won.loc[winner_name, loser_name] = split_value
# Ensure indexes and labels are in the same order
total_money_won.sort_index(axis=0, inplace=True)
total_money_won.sort_index(axis=1, inplace=True)
total_money_won.fillna(0, inplace=True)


total_money_difference = total_money_won.copy(deep=True)
for i in range(0, len(total_money_won.columns)):
    for j in range(0, len(total_money_won)):
        # print(f'{i}, {j}')
        total_money_difference.iloc[i, j] = total_money_won.iloc[i, j] - total_money_won.iloc[j, i]

total_money_difference

,Asher @ 19TVcwV9oC,BJ @ ZvsiqtjDfV,Cody @ HO1vaicBKi,Ernie @ JzwM-gY3G_,Gautam @ XR0UMSLq50,Ravi @ xn9mvyMGGz,Tony @ I8y9iW09Dh,Tuts 2021 @ jmPBr71waH,trav 2.0 @ KSAenvQzCz,trav @ KSAenvQzCz
Asher @ 19TVcwV9oC,0.0,-189.0,0.0,453.0,-504.0,110.0,-220.0,40.0,0.0,20.0
BJ @ ZvsiqtjDfV,189.0,0.0,0.0,180.0,-330.0,-248.0,-1525.0,305.0,0.0,-40.0
Cody @ HO1vaicBKi,0.0,0.0,0.0,10.0,140.0,60.0,-790.0,-60.0,970.0,120.0
Ernie @ JzwM-gY3G_,-453.0,-180.0,-10.0,0.0,-120.0,-498.0,-229.0,-490.0,0.0,-20.0
Gautam @ XR0UMSLq50,504.0,330.0,-140.0,120.0,0.0,25.0,-397.0,110.0,0.0,935.0
Ravi @ xn9mvyMGGz,-110.0,248.0,-60.0,498.0,-25.0,0.0,-90.0,165.0,0.0,-10.0
Tony @ I8y9iW09Dh,220.0,1525.0,790.0,229.0,397.0,90.0,0.0,100.0,30.0,30.0
Tuts 2021 @ jmPBr71waH,-40.0,-305.0,60.0,490.0,-110.0,-165.0,-100.0,0.0,0.0,-35.0
trav 2.0 @ KSAenvQzCz,0.0,0.0,-970.0,0.0,0.0,0.0,-30.0,0.0,0.0,0.0
trav @ KSAenvQzCz,-20.0,40.0,-120.0,20.0,-935.0,10.0,-30.0,35.0,0.0,0.0


In [25]:
total_money_won

,Asher @ 19TVcwV9oC,BJ @ ZvsiqtjDfV,Cody @ HO1vaicBKi,Ernie @ JzwM-gY3G_,Gautam @ XR0UMSLq50,Ravi @ xn9mvyMGGz,Tony @ I8y9iW09Dh,Tuts 2021 @ jmPBr71waH,trav 2.0 @ KSAenvQzCz,trav @ KSAenvQzCz
Asher @ 19TVcwV9oC,0.0,285.0,20.0,463.0,110.0,130.0,20.0,40.0,0.0,20.0
BJ @ ZvsiqtjDfV,474.0,0.0,80.0,180.0,375.0,120.0,340.0,830.0,0.0,90.0
Cody @ HO1vaicBKi,20.0,80.0,0.0,50.0,370.0,180.0,330.0,30.0,970.0,120.0
Ernie @ JzwM-gY3G_,10.0,0.0,40.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0
Gautam @ XR0UMSLq50,614.0,705.0,230.0,120.0,0.0,130.0,560.0,160.0,0.0,1000.0
Ravi @ xn9mvyMGGz,20.0,368.0,120.0,498.0,105.0,0.0,20.0,215.0,0.0,0.0
Tony @ I8y9iW09Dh,240.0,1865.0,1120.0,229.0,957.0,110.0,0.0,310.0,30.0,40.0
Tuts 2021 @ jmPBr71waH,0.0,525.0,90.0,530.0,50.0,50.0,210.0,0.0,0.0,40.0
trav 2.0 @ KSAenvQzCz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trav @ KSAenvQzCz,0.0,130.0,0.0,20.0,65.0,10.0,10.0,75.0,0.0,0.0
